<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/11_cnn_pytorch/05_abn_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention Branch Network (ABN)

---

## 目的
ABNの構造を理解する.

ABNを用いてCIFAR-10データセットに対する物体認識を行う．

## Attention Branch Network (ABN)
Attention Branch Network (ABN)[1]とは，Attention機構により推論時における注視領域の可視化と，画像認識の高精度化ができるネットワークです．
02_CAM.ipynbで使用したClass Activation Mapping (CAM)[2] は，CNNが推論時に着目した領域をAttention mapとして可視化できる一方で，最後の畳み込み層の後のGlobal Average Pooling (GAP)[3] によって特徴マップの空間的な情報が失われることによる認識精度の低下が問題となっていました．
ABNでは，ネットワークの判断根拠であるAttention mapを認識処理に用いることで説明性の高いAttention mapの獲得と認識精度の高精度化を実現しました．
ABNは，Feature extractor，Attention branch，Perception branchの3つのモジュールから構成されています．
Feature extractorは，複数の畳み込み層が含まれており，入力から特徴マップを獲得します．
Attention branchは，特徴マップを受け取り，クラス識別及びAttention mapを生成します．
Attention branchにより生成されたAttention mapは，Attention機構によりFeature extractorから得られた特徴マップ全体に対して重み付けします．
この重み付けした特徴マップを用いてPerception branchにより最終的な各クラスにおける確率を出力します．
これにより，画像認識に有効な領域に着目して学習することができます．
学習時は，以下の式よりAttention branchとPerception branchの学習誤差を用いてネットワークを学習します．

$$L_{all}({\bf x}_i) = L_{att}({\bf x}_i) + L_{per}({\bf x}_i)$$

ここで，${\rm x}_i$は$i$番目の入力画像，$L_{att}({\bf x}_i)$はAttention branch の学習誤差，$L_{per}({\bf x}_i)$ はPerception branchの学習誤差を示します．
ABNは，Attention branchをベースラインのネットワークに追加することで，様々な画像認識タスクに応用することができます．

### Attention branch
Attention branchは，畳み込み層とGAPから構成されており，Attention mapを出力するモジュールです．Attention branchでは，GAPの2層前の特徴マップに対し，$1×1$の重みフィルタを畳み込みます．
その後，Sigmoid関数により0から1の範囲に正規化することでAttention mapを獲得します．

### Attention機構
Attention機構とは，認識に有益な特徴量に対して重み付けして学習することで，ネットワークの汎化性能を向上させる方法です．
ABNでは，Attention branchにより生成されたAttention mapをAttention機構によりFeature extractorから得られた特徴マップ全体に対して重み付けします．
また，重み付け後の特徴マップに重み付け前の特徴マップを加算することで，特徴マップの消失を抑制し，Attention mapを認識に効率的に反映させることができます．
ABNでは，以下の式よりFeature extractorから得られた特徴マップに対してAttention mapを重み付けします．

$$f'({\rm x}_i)=(1+M({\rm x}_i))\cdot f({\rm x}_i)$$

ここで，$M({\rm x}_i)$は$i$番目の入力画像に対するAttention mapを示しています．

### Perception branch
Perception branchは，Attention機構により重み付けされた特徴マップを入力し，最終的な認識結果を出力します．
<img src="https://drive.google.com/uc?export=view&id=1AWFGV9j9I4VVlaus79Iwr2OzX60gR-PR" width=100%>




## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．
今回は，機械学習ライブラリであるPytorchを使用します．
PyTorchとは，Python向けのオープンソース機械学習ライブラリで，Facebookに開発されました．

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import torchsummary

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．
下記のコードを実行してGPU情報を確認します． GPUの確認を行うためには，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPytorchで行うことが可能です． CPUとなっている場合は，上記に記載している手順にしたがって，設定を変更してください．

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

下記のコードを実行してGPU情報を確認します．



In [ ]:
!nvidia-smi

## 使用するデータセット

### データセット
今回の物体認識では，CIFAR-10データセットを使用します．CIFAR-10データセットは，飛行機や犬などの10クラスの物体が表示されている画像から構成されたデータセットです．

![CIFAR10_sample.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/176458/b6b43478-c85f-9211-7bc6-227d9b387af5.png)

### データセットのダウンロードと読み込み
実験に使用するCIFAR-10データセットを読み込みます．
１回の誤差を算出するデータ数 (ミニバッチサイズ) は，64とします．
まず，CIFAR-10データセットをダウンロードします．
次に，ダウンロードしたデータセットを読み込みます．
学習には，大量のデータを利用しますが，それでも十分ではありません． そこで，データ拡張 (data augmentation) により，データのバリエーションを増やします． 一般的な方法は，画像の左右反転，明るさ変換などです． また，画像の一部にマスク処理をかけるRandom Erasingも行います．

In [ ]:
batch_size = 64

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    transforms.RandomErasing(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=20)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=20)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### CIFAR-10データセットの表示
CIFAR-10データセットに含まれる画像を表示してみます．
ここでは，matplotlibを用いて複数の画像を表示させるプログラムを利用します．
学習データは5万枚，1つのデータサイズは3x32x32の画像のような形式となっています． これは32x32ピクセルのカラー画像という意味になります．



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    npimg = img.numpy()
    npimg = ((npimg.transpose((1,2,0))  * [0.2023, 0.1994, 0.2010]) + [0.4914, 0.4822, 0.4465])  # unnormalize
    plt.imshow(npimg)
    plt.show()

# get some random training images
dataiter = iter(testloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images[0:4]))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

## ネットワークモデルの定義
次に，ABNを定義します．
今回は，ResNetにABNを導入したモデル (ResNet-ABN) を使用します．ResNet-ABNの層の深さ (depth) は，20層に設定します．

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import math

    
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True),
            nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(planes),
        )
        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.convs(x)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_planes, planes, kernel_size=1, bias=False),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True),
            nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True),
            nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.expansion * planes),
        )
        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.convs(x)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class ResNet_ABN(nn.Module):
    def __init__(self, depth, num_classes=10):
        super(ResNet_ABN, self).__init__()
        # Model type specifies number of layers for CIFAR-10 model
        block_name = 'Bottleneck' if depth >=44 else 'BasicBlock'
        if block_name.lower() == 'basicblock':
            assert (depth - 2) % 6 == 0, 'When use basicblock, depth should be 6n+2, e.g. 20, 32, 44'
            n = (depth - 2) // 6
            block = BasicBlock
        elif block_name.lower() == 'bottleneck':
            assert (depth - 2) % 9 == 0, 'When use bottleneck, depth should be 9n+2, e.g. 47, 56, 110, 1199'
            n = (depth - 2) // 9
            block = Bottleneck
        else:
            raise ValueError('block_name shoule be Basicblock or Bottleneck')

        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, n, down_size=True)
        self.layer2 = self._make_layer(block, 32, n, stride=2, down_size=True)

        self.att_layer3 = self._make_layer(block, 64, n, stride=1, down_size=False)
        self.bn_att = nn.BatchNorm2d(64 * block.expansion)
        self.att_conv   = nn.Conv2d(64 * block.expansion, num_classes, kernel_size=1, padding=0,
                               bias=False)
        self.bn_att2 = nn.BatchNorm2d(num_classes)
        self.att_conv2  = nn.Conv2d(num_classes, num_classes, kernel_size=1, padding=0,
                               bias=False)
        self.att_conv3  = nn.Conv2d(num_classes, 1, kernel_size=3, padding=1,
                               bias=False)
        self.bn_att3 = nn.BatchNorm2d(1)
        self.att_gap = nn.AvgPool2d(16)
        self.sigmoid = nn.Sigmoid()

        self.layer3 = self._make_layer(block, 64, n, stride=2, down_size=True)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1, down_size=True):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        if down_size:
            self.inplanes = planes * block.expansion
            for i in range(1, blocks):
                layers.append(block(self.inplanes, planes))

            return nn.Sequential(*layers)
        else:
            inplanes = planes * block.expansion
            for i in range(1, blocks):
                layers.append(block(inplanes, planes))

            return nn.Sequential(*layers)


    def forward(self, x):

        # feature extractor
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)    # 32x32

        x = self.layer1(x)  # 32x32
        x = self.layer2(x)  # 16x16

        # attention branch
        ax = self.bn_att(self.att_layer3(x))
        ax = self.relu(self.bn_att2(self.att_conv(ax)))
        bs, cs, ys, xs = ax.shape
        self.att = self.sigmoid(self.bn_att3(self.att_conv3(ax)))
        ax = self.att_conv2(ax)
        ax = self.att_gap(ax)
        ax = ax.view(ax.size(0), -1)

        # perception branch
        rx = x * self.att
        rx = rx + x
        rx = self.layer3(rx)  # 8x8
        rx = self.avgpool(rx)
        rx = rx.view(rx.size(0), -1)
        rx = self.fc(rx)

        return ax, rx, self.att

depth = 20 # e.g. 20, 32, 44, 47, 56, 110, 1199
model = ResNet_ABN(depth)
model.to(device)

## 損失関数と最適化手法の定義
学習に使用する損失関数と最適化手法を定義します．
今回は，分類問題を扱うため，クロスエントロピー誤差を計算するための`CrossEntropyLoss`を`criterion`として定義します．
最適化手法には，確率的勾配降下法 (stochastic gradient descent: SGD) を用いて学習します．
最後に，定義したネットワークの詳細情報を`torchsummary.summary()`関数を用いて表示します．


In [ ]:
import torch.optim as optim
from torch.optim import SGD, lr_scheduler

criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9,  weight_decay=5e-4)
#scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[150, 225], gamma=0.1) # 50 < 75

# モデルの情報を表示
torchsummary.summary(model, (3, 32, 32))

## 学習
読み込んだCIFAR-10データセットと作成したネットワークを用いて，学習を行います．
今回は，学習エポック数を15とします．
1エポック学習するごとに学習したモデルを評価し，最も認識精度の高いモデルが保存されます．
各更新において，学習用データと教師データをそれぞれ`inputs`と`targets`とします．
学習モデルに`inputs`を与えて，Attention branchの出力，Perception branchの出力及びAttention mapを取得します．
Attention branchの出力及びPerception branchの出力と教師ラベル`targets`との誤差を`criterion`で算出します．
また，認識精度も算出します．
そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．
認識精度も同時に計算して，`print`関数で学習経過における誤差や認識精度を表示します．

In [ ]:
epoch_num = 15
best_acc = 0  # best test accuracy

for epoch in range(1, epoch_num+1):
    #scheduler.step()
    train_running_loss = 0.0
    train_running_acc = 0.0

    # training
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    count = 0
    for image, label in trainloader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
        optimizer.zero_grad()
        att_outputs, per_outputs, _ = model(image)
        att_loss = criterion(att_outputs, label)
        per_loss = criterion(per_outputs, label)
        loss = att_loss + per_loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = per_outputs.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()
        # print statistics
        train_running_loss += loss
        train_running_acc += 100.*correct/total
        count += 1

    print('[Epoch %d] Train Loss: %.5f | Train Acc: %.3f%%'
                  % (epoch, train_loss/count, train_running_acc/count))

    # testing
    model.eval() 
    with torch.no_grad():
        test_running_loss = 0.0
        test_running_acc = 0.0
        test_loss = 0
        correct = 0
        total = 0
        count = 0
        for image, label in testloader:
          if use_cuda:
              image = image.cuda()
              label = label.cuda()
          _, outputs, attention = model(image)
          loss = criterion(outputs, label)

          test_loss += loss.item()
          _, predicted = outputs.max(1)
          total += label.size(0)
          correct += predicted.eq(label).sum().item()

          # print statistics
          test_running_loss += loss
          test_running_acc += 100.*correct/total
          count += 1

        print('Test Loss: %.5f | Test Acc: %.3f%%'
                      % (test_loss/count, test_running_acc/count))
        
    # save model
    if test_running_acc/count > best_acc:
        best_acc = max(test_running_acc/count, best_acc)
        PATH = './cifar_net.pth'
        torch.save(model.state_dict(), PATH)
    
print('Finished Training')

##テスト
学習したネットワークのテストデータに対する認識精度の確認を行います．
まず，学習したネットワークを評価するために保存したモデルをロードします．

In [ ]:
depth = 20 #e.g. 20, 32, 44, 47, 56, 110, 1199
model = ResNet_ABN(depth)
model.to(device)
PATH = './cifar_net.pth'
model.load_state_dict(torch.load(PATH))

次に，学習したネットワークを用いて，テストデータに対する認識精度の確認を行います．
`model.eval()`を適用することで，ネットワーク演算を評価モードへ変更します． これにより，学習時と評価時で挙動が異なる演算（dropout等）を変更することが可能です． また，`torch.no_grad()`を適用することで，学習時には必要になる勾配情報を保持することなく演算を行います．

In [ ]:
model.eval()
running_loss = 0.0
running_acc = 0.0
test_loss = 0
correct = 0
total = 0
count = 0
with torch.no_grad():
    for image, label in testloader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
        _, outputs, attention = model(image)
        loss = criterion(outputs, label)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        # print statistics
        running_loss += loss
        running_acc += 100.*correct/total
        count += 1

    print('Test Loss: %.5f | Test Acc: %.3f%%'
                  % (test_loss/count, running_acc/count))

## Attention mapの可視化
Attention mapを可視化して，ネットワークの判断根拠を確認してみます．
再度，実行することで他のテストサンプルに対するAttention mapを可視化することができます．
pred (prediction) は認識結果，conf (confidence) は認識結果に対する信頼度を示しています．

In [ ]:
import cv2
from ipykernel import kernelapp as app

testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=True, num_workers=20)

classes_list = ['plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

softmax = nn.Softmax(dim=1)

def min_max(x, axis=None):
    x_min = x.min(axis=axis, keepdims=True)
    x_max = x.max(axis=axis, keepdims=True)
    return (x - x_min) / (x_max - x_min)

with torch.no_grad():
    for image, label in testloader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
        _, outputs, attention = model(image)
        outputs = softmax(outputs)
        conf_data = outputs.data.topk(1, 1, True, True)
        _, predicted = outputs.max(1)
        c_att = attention.data.cpu()
        c_att = c_att.numpy()
        d_inputs = image.data.cpu()
        d_inputs = d_inputs.numpy()

        in_b, in_c, in_y, in_x = image.shape
        v_list = []
        att_list = []
        for item_img, item_att in zip(d_inputs, c_att):
            v_img = ((item_img.transpose((1, 2, 0)) * [0.2023, 0.1994, 0.2010]) + [0.4914, 0.4822, 0.4465]) * 255
            #v_img = v_img[:, :, ::-1]
            resize_att = cv2.resize(item_att[0], (in_x, in_y))
            #resize_att = min_max(resize_att)
            resize_att *= 255.

            v_img = np.uint8(v_img)
            resize_att = np.uint8(resize_att)
            jet_map = cv2.applyColorMap(resize_att, cv2.COLORMAP_JET)
            jet_map = cv2.addWeighted(v_img, 0.6, jet_map, 0.4, 0)
            v_list.append(v_img)
            att_list.append(jet_map)
        break

# Show attention map
cols = 8
rows = 1

fig = plt.figure(figsize=(14, 3.0))
plt.title('Input image')
plt.axis("off")
for r in range(rows):
    for c in range(cols):
        cls = label[c].item()
        ax = fig.add_subplot(r+1, cols, c+1)
        plt.title('{}'.format(classes_list[cls]))
        ax.imshow(v_list[cols * r + c])
        ax.set_axis_off()
plt.show()

fig = plt.figure(figsize=(14, 3.5))
plt.title('Attention map')
plt.axis("off")
for r in range(rows):
    for c in range(cols):
        pred = predicted[c].item()
        conf = conf_data[0][c].item()
        ax = fig.add_subplot(r+1, cols, c+1)
        ax.imshow(att_list[cols * r + c])
        plt.title('pred: {}\nconf: {:.2f}'.format(classes_list[pred], conf))
        ax.set_axis_off()
plt.show()

## 課題
1. エポック数やミニバッチサイズを変えて実験し，認識精度とAttention mapの変化を確認しましょう．

In [ ]:
#ここにコードを書く

# 参考文献
- [1] H. Fukui, T. Hirakawa, T. Yamashita, and
H. Fujiyoshi, "Attention branch network: Learning of
attention mechanism for visual explanation". In 2019 IEEE
Conference on Computer Vision and Pattern Recognition,
pp. 10705–10714, 2019.

- [2] B. Zhou, A. Khosla, A. Lapedriza, A. Oliva,
and A. Torralba, "Learning deep features for discriminative
localization". In 2016 IEEE Conference on Computer
Vision and Pattern Recognition, pp. 2921–2929, 2016.

- [3] M. Lin, Q. Chen, and S. Yan, "Network in network".
In 2nd International Conference on Learning Representations,
Banff, AB, Canada, April 14-16, 2014, Conference
Track Proceedings, 2014.